In [0]:
# Create DataFrame from data source - csv file
customerDF = spark.read.load("/FileStore/tables/SparkSQL/customers.txt", format="csv", sep="\t", inferSchema="true", header="true")

In [0]:
# Different types of operations on DataFrames
customerDF.count()

In [0]:
customerDF.printSchema()

In [0]:
customerDF.select("customer_name","customer_city").show()

In [0]:
customerDF.select(customerDF.customer_name,customerDF.customer_city).show(n=5)

In [0]:
customerDF.select(customerDF['customer_name'], customerDF['customer_city']).show(5)

In [0]:
customerDF.where(customerDF['customer_state'] == 'CA').show(5)

In [0]:
df2=customerDF.where(customerDF['customer_state'] == 'CA')
type(df2)

In [0]:
df2.show(5)

In [0]:
customerDF.groupBy("customer_state").count().show(5)

In [0]:
customerDF.groupBy("customer_state").count().orderBy("customer_state").show(5)

In [0]:
# Create temp view for running SQL queries on the dataframe
customerDF.createOrReplaceTempView("customers")

In [0]:
# SQL can be run on DataFrames that are registered as temp views.
spark.sql("SELECT customer_state, count(*) as state_count FROM customers \
GROUP BY customer_state HAVING state_count>=50").show()

In [0]:
cStateCount50 = spark.sql("SELECT customer_state, count(*) as state_count FROM customers \
GROUP BY customer_state HAVING state_count>=50")
type(cStateCount50)

In [0]:
cStateCount50.show()

In [0]:
cStateCount50.printSchema()

In [0]:
cStateCount50.rdd.getNumPartitions()

In [0]:
cStateCount50.write.save("/FileStore/tables/SparkSQL/cStateOutput1.parquet")

# We can use the above since parquet is the default format used by SparkSQL.
# Alternatively the following syntax is also allowed.
# cStateCount50.write.parquet("/FileStore/tables/SparkSQL/cStateOutput2.parquet")

In [0]:
cStateCount50.write.save("/FileStore/tables/SparkSQL/cStateOutput1.json", format="json")

# Alternatively the following syntax is also allowed.
# cStateCount50.write.json("/FileStore/tables/SparkSQL/cStateOutput2.json")

In [0]:
dbutils.fs.head("dbfs:/FileStore/tables/SparkSQL/cStateOutput1.json/part-00000-tid-705762262804892611-3af422af-a723-4499-86ff-e31b51806351-269-1-c000.json")

In [0]:
cStateCount50.write.save("/FileStore/tables/SparkSQL/cStateOutput1.csv", format="csv", sep="\t", header="true")
# Alternatively the following syntax is also allowed.
# cStateCount50.write.csv("/FileStore/tables/SparkSQL/cStateOutput2.csv", sep="\t", header="true")

In [0]:
dbutils.fs.head("dbfs:/FileStore/tables/SparkSQL/cStateOutput1.csv/part-00000-tid-1989546960943500210-b1fe2feb-1556-42ef-b864-94ca3f8d486a-271-1-c000.csv")

In [0]:
# Aggregatons like sum, mon, max, avg can be done as shown later below
from pyspark.sql.functions import *

In [0]:
customerDF.groupBy("customer_state").count().orderBy(col("customer_state").desc()).show(5)

In [0]:
customerDF.groupBy("customer_state").count().sort("customer_state").show(5)

In [0]:
customerDF.groupBy("customer_state").count().sort(col("customer_state").desc()).show(5)

In [0]:
productDF = spark.read.load("/FileStore/tables/SparkSQL/products.json", format="json")
productDF.count()

# Alternatively the following syntax is also allowed
# productDF = spark.read.json("products.json")

In [0]:
productDF.printSchema()

In [0]:
productDF.select("product_name").show(5)

In [0]:
productDF.select("product_name").show(n=5,truncate=False)

In [0]:
productDF.select("product_name").show(5,False)

In [0]:
productDF.select(productDF['product_name'], productDF['product_category'], productDF['product_price']).show(5)

In [0]:
productDF.filter(productDF['product_price'] > 200.00).show(5)

In [0]:
productDF.groupBy("product_category").count().show(5)

In [0]:
# Create temp view for running SQL queries on the dataframe
productDF.createOrReplaceTempView("products")

In [0]:
# SQL queries can be run on the DataFrames that have been registered as a table.
prd200 = spark.sql("SELECT category_id, product_category, count(*) as prdcount FROM products WHERE product_price>200 GROUP BY category_id, product_category ORDER BY product_category")
type(prd200)

In [0]:
prd200.show()

In [0]:
custlist200 = spark.sql("SELECT a.customer_name, b.product_category, count(*) as prdcount \
FROM customers a INNER JOIN products b ON a.customer_id=b.customer_id WHERE b.product_price>200.00 \
GROUP BY a.customer_name, b.product_category HAVING prdcount>1")
type(custlist200)

In [0]:
custlist200.printSchema()

In [0]:
custlist200.count()

In [0]:
custlist200.show(5)

In [0]:
productDF.groupBy("product_category").max("product_price").show()

In [0]:
productDF.groupBy("product_category").avg("product_price").show()

In [0]:
productDF.groupBy("product_category").agg(avg("product_price").alias("avg_price"),).show(truncate=False)

In [0]:
productDF.groupBy("product_category").agg(round(avg("product_price"),2).alias("avg_price")).show(truncate=False)


In [0]:
productDF.groupBy("product_category").\
    agg(\
        count("product_price").alias("product_count"),\
        avg("product_price").alias("avg_price"),\
        max("product_price").alias("max_price"),\
        min("product_price").alias("min_price")\
    ).\
    show(truncate=False)


In [0]:
productDF.groupBy("product_category").\
    agg(\
        count("product_price").alias("product_count"),\
        round(avg("product_price"),2).alias("avg_price"),\
        max("product_price").alias("max_price"),\
        min("product_price").alias("min_price")\
    ).\
    show(truncate=False)


In [0]:
# withColumn
## cast
prodDF2=productDF.withColumn('customer_id',col('customer_id').cast("Integer"))
prodDF2.printSchema()

In [0]:
# withColumn
## add a new column with constant
prodDF3=prodDF2.withColumn('discount',lit(10.0))
prodDF3.printSchema()
prodDF3.show(5)

In [0]:
# withColumn
## add a new column calculated with existing columns
prodDF4=prodDF3.withColumn('discounted_price',col('product_price')*(100-col('discount'))/100)
prodDF4.printSchema()
prodDF4.show(5)

In [0]:
# withColumnRenamed
prodDF5=prodDF4.withColumnRenamed('customer_id','prodcust_id')
prodDF5.printSchema()
prodDF5.show(5)

In [0]:
# Using alias to rename a column
prodDF5a = prodDF4.select(col('category_id'),col('customer_id').alias('prodcust_id'),\
                          col('product_category'),col('product_name'),col('product_price'),col('product_quantity'),\
                         col('salestxn_id'),col('discount'),col('discounted_price'))

prodDF5a.printSchema()
prodDF5a.show(5)

In [0]:
# join
newDF=customerDF.join(prodDF5,customerDF.customer_id==prodDF5.prodcust_id,'outer')
newDF.printSchema()

In [0]:
newDF.select('customer_id','customer_name','customer_city','prodcust_id','product_name','product_price','product_quantity').where(newDF.customer_state=='IL').show()

In [0]:
# df.na.fill on columns with a string type data
newDF2=newDF.na.fill(value='NA',subset=['product_name','product_category'])
newDF2.printSchema()
newDF2.select('customer_id','customer_name','customer_city','product_category','prodcust_id','product_name','product_price','product_quantity').where(newDF.customer_state=='IL').show()

In [0]:
# df.na.fill on columns with an integer type data
newDF3=newDF2.na.fill(value=0,subset=['prodcust_id','category_id'])

In [0]:
newDF3.printSchema()

In [0]:
newDF3.select('customer_id','customer_name','customer_city','category_id','product_category','prodcust_id',\
              'product_name','product_price','product_quantity').where(newDF.customer_state=='IL').show()

In [0]:
newDF.columns

In [0]:
type(newDF.columns)

In [0]:
len(newDF.columns)